In [ ]:
import pandas as pd
import numpy as np
import csv
import xlsxwriter
from csv import reader, writer
from openpyxl import Workbook
import datetime as dt

In [ ]:
#Import mobile route info
cs2 = "z_MAY_ROUTES.csv"
mobile = pd.read_csv(cs2)
#mobile.head(2)

In [ ]:
#Zones & Messages > Customer Visits > Select All assets > View by Asset
#Pull Details Advanced Report
#Unhide Data Tab
#Delete header info from Data tab
#Change column Q duration to 00:00:00 Time format
#Save as CSV
cs1 = "customer_visitsz.csv"
df = pd.read_csv(cs1)
#df.head(2)

In [ ]:
df2 = df.drop(columns=['.Device.DeviceId', '.Device.DeviceComment', '.Device.DeviceGroup', '.Driver.UserFirstName', '.Driver.UserLastName', '.Driver.UserId', '.Driver.UserComment', '.Driver.DriverGroup', '.ExceptionRule.ExceptionRuleId', '.ExceptionRule.ExceptionRuleComment', '.StopZone.ZoneZoneTypes', 'ExceptionDetailLongitude', 'ExceptionDetailLatitude'])
#df2.head(2)

In [ ]:
df2.rename({'.Device.DeviceName':'truck', '.Driver.UserName':'driver', '.ExceptionRule.ExceptionRuleName':'zone', '.StopZone.ZoneExternalReference':'ship_to', 'ExceptionTripDistance':'distance', 'ExceptionDetailStartTime':'arrived', 'ExceptionDuration':'duration'},axis =1, inplace = True)
#df2.head(1)

In [ ]:
df2['date'] = pd.to_datetime(df2['arrived']).dt.date
#df2.head()

In [ ]:
df3 = df2.drop(columns=['arrived'])
#df3.head(2)

In [ ]:
df3['duration'] = pd.to_timedelta(df3['duration'])
#df3.head(2)

In [ ]:
df3['day_hours'] = df3['duration'].dt.days * 24
#df3.head()

In [ ]:
df3['hours'] = df3['duration'].dt.components['hours']
#df3.head()

In [ ]:
df3['minutes'] = df3['duration'].dt.components['minutes'] / 60
#df3.head()

In [ ]:
df3['total_hours'] = df3['day_hours'] + df3['hours'] + df3['minutes']
#df3.head()

In [ ]:
df4 = df3.drop(columns=['duration', 'day_hours', 'hours', 'minutes'])
#df4.head(2)

In [ ]:
df5 = df4.groupby(['date', 'ship_to', 'zone', 'driver', 'truck'], as_index = False).sum()
#df5.head(10)

In [ ]:
#df4.to_csv("customer_visit_detail.csv", header=True, index=False)

# ----------

## Summary

In [ ]:
summary = df5.groupby(['ship_to', 'zone']).agg({'distance':['mean', 'min', 'max'], 'total_hours':['mean', 'min', 'max']})
#summary.head(10)

In [ ]:
#summary.to_csv("customer_summary.csv")

In [ ]:
with pd.ExcelWriter('customer_visits.xlsx', engine='xlsxwriter') as writer:    
    #Write each dataframe to a different worksheet.
    df4.to_excel(writer, sheet_name='Customer Visit Detail', header=True, index=False)
    summary.to_excel(writer, sheet_name='Customer Visit Summary', header=True, index=True)

# --------------

## Merge GeoTab and SLX/Trinium info

In [ ]:
#mobile.head(1)

In [ ]:
mobile2 = mobile.drop(columns=['type', 'total_time', 'paylocity_time'])
#mobile2.head(1)

In [ ]:
mobile3 = mobile2.rename(columns={'shift_date':'date', 'id':'slx_id', 'driver':'slx_driver'})
#mobile3.head(1)

In [ ]:
mobile3['date'] = pd.to_datetime(mobile3['date']).dt.date
#mobile3.head(2)

In [ ]:
merged = pd.merge(mobile3, df5, on=['date', 'ship_to', 'truck'], how='left')
merged.head()

In [ ]:
#Drop service trucks
merged2 = merged[~merged.truck.str.contains('Service')]
merged2.head()

In [ ]:
#Drop internal sites
merged3 = merged2[~merged2.ship_to.str.startswith('10000')]
#merged3.head(2)

In [ ]:
merged4 = merged3[merged3['slx_id'].notna()]
#merged4.head()

In [ ]:
merged4['profit_per_hour'] = merged4['profit']/merged4['total_hours']
#merged4.head(3)


In [ ]:
smart = merged4.groupby(['ship_to', 'zone']).agg({'ship_to':['count'],'distance':['min','mean', 'max'],'profit_per_hour':['min','mean', 'max'],'margin':['min','mean', 'max'],'gallons':['min','mean', 'max'], 'total_hours':['min','mean', 'max']})
#smart.head(10)

In [ ]:
with pd.ExcelWriter('Geotab_SLX_Report.xlsx', engine='xlsxwriter') as writer:    
    #Write each dataframe to a different worksheet.
    merged4.to_excel(writer, sheet_name='Smartlogix Detailed Report', header=True, index=True)
    smart.to_excel(writer, sheet_name='Averages', header=True, index= True)

In [ ]:
merged4.to_csv("fxz.csv", header=True, index=False)